In [20]:
# lstm autoencoder to recreate a timeseries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
'''
A UDF to convert input data into 3-D
array as required for LSTM network.
'''

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [2]:
timeseries = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                       [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3]]).transpose()

In [3]:
timeseries

array([[0.1  , 0.001],
       [0.2  , 0.008],
       [0.3  , 0.027],
       [0.4  , 0.064],
       [0.5  , 0.125],
       [0.6  , 0.216],
       [0.7  , 0.343],
       [0.8  , 0.512],
       [0.9  , 0.729]])

In [4]:
timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]

In [5]:
timesteps = 3
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

In [6]:
timesteps = 3
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

n_features = 2
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)



In [22]:
model = Sequential()
model.add(Bidirectional(LSTM(10, activation='relu', input_shape=(timesteps,n_features), return_sequences=False)))
model.add(RepeatVector(timesteps))
model.add(LSTM(10, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# model.summary()

In [23]:
# fit model
model.fit(X, X, epochs=300, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[0.32  0.057]
  [0.444 0.078]
  [0.502 0.113]]

 [[0.371 0.083]
  [0.516 0.133]
  [0.582 0.193]]

 [[0.437 0.132]
  [0.61  0.234]
  [0.687 0.335]]

 [[0.527 0.217]
  [0.73  0.381]
  [0.803 0.502]]

 [[0.644 0.318]
  [0.857 0.545]
  [0.936 0.667]]]
---Actual---
[[[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]


In [24]:
from keras.models import Model
from keras.utils import plot_model
modelEnc = Model(inputs=model.inputs, outputs=model.layers[0].output)

In [25]:
# plot_model(model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat = modelEnc.predict(X[0][np.newaxis, :])
print(yhat.shape)
print(yhat)

(1, 20)
[[0.28707868 0.         0.16373603 0.02692483 0.18916713 0.
  0.24376826 0.02470064 0.1955162  0.         0.11040504 0.01515147
  0.         0.         0.00112201 0.         0.06477787 0.20481361
  0.08068737 0.        ]]


In [26]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 20)                1040      
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 3, 20)             0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 3, 10)             1240      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 3, 2)              22        
Total params: 2,302
Trainable params: 2,302
Non-trainable params: 0
_________________________________________________________________
